# Multiple Linear Regression
First we will use multiple linear regression to predict the rating based on the features such as region and roast.

## Uploading the data and creating a train test split

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
## first load the data
coffee = pd.read_csv('../data/one_hot_coffee.csv')
coffee = coffee.copy()

In [4]:
## next perform the train test split
coffee_train, coffee_test = train_test_split(coffee,
                                            shuffle=True,
                                            random_state=47,
                                            test_size = .2)

## Exploration and Creating a Baseline
Here are some exploratory OLS metrics with code courtesy of our mentor Andrew Castillo.

In [12]:
import statsmodels.api as sm

In [13]:
X_sm = sm.add_constant(coffee_train[predictors])
mlr = sm.OLS(coffee_train.rating,X_sm)
mlr.fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     117.3
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               0.00
Time:                        16:33:22   Log-Likelihood:                -9602.3
No. Observations:                3741   AIC:                         1.924e+04
Df Residuals:                    3722   BIC:                         1.936e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     75.7545      0.086    876.875      0.000      75.585      75.924
region_africa_arabia       1.8811      0.146     12.862      0.000       1.594       2.168
region_caribbean          -1.7335      0.752     -2.306      0.021      -3.207      -0.260
region_central_america     0.4314      0.178      2.428      0.015       0.083       0.780
region_hawaii              0.9310      0.393      2.367      0.018       0.160       1.702
region_asia_pacific        0.2260      0.211      1.070      0.285      -0.188       0.640
region_south_america       0.0784      0.210      0.373      0.709      -0.333       0.490
type_espresso              1.7595      0.155     11.364      0.000       1.456       2.063
type_organic              -0.0346      0.239     -0.144      0.885      -0.504       0.435
type_fair_trade            0.0968      0.294      0.329      0.742      -0.481       0.674
type_decaffeinated         0.2582      0.535      0.483      0.629      -0.791       1.307
type_pod_capsule          -1.8647      0.300     -6.220      0.000      -2.452      -1.277
type_blend                 0.2236      0.192      1.163      0.245      -0.153       0.601
type_estate                0.6074      0.181      3.357      0.001       0.253       0.962
Dark                       9.5367      0.197     48.392      0.000       9.150       9.923
Light                     15.5745      0.169     91.923      0.000      15.242      15.907
Medium                    14.0938      0.102    138.515      0.000      13.894      14.293
Medium-Dark               11.5819      0.125     92.782      0.000      11.337      11.827
Medium-Light              15.4516      0.103    149.507      0.000      15.249      15.654
Very Dark                  9.5162      0.230     41.303      0.000       9.064       9.968
==============================================================================
Omnibus:                     1929.847   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21384.432
Skew:                          -2.206   Prob(JB):                         0.00
Kurtosis:                      13.850   Cond. No.                     6.29e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.35e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

For a baseline model to compare our methods to, we will use a constant prediction of the the average rating for the coffee.

In [5]:
## make a baseline, which we will simply take to be the mean of the ratings
baseline = coffee_train['rating'].mean()
print(baseline)

90.43704891740177


## Creating MLR Model and running cross validation

In [19]:
## importing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold

In [20]:
## creating a list of predictor variables
predictors = ['region_africa_arabia', 'region_caribbean',
       'region_central_america', 'region_hawaii', 'region_asia_pacific',
       'region_south_america', 'type_espresso', 'type_organic',
       'type_fair_trade', 'type_decaffeinated', 'type_pod_capsule',
       'type_blend', 'type_estate', 'Dark', 'Light', 'Medium', 'Medium-Dark', 'Medium-Light', 'Very Dark']

In [21]:
## perform cross validation for the linear regression model

splits = 5
kfold = KFold(n_splits=splits, shuffle=True, random_state=413)

reg = LinearRegression(copy_X = True)
mse = np.empty(splits)
mae = np.empty(splits)

i = 0

for train_index, test_index in kfold.split(coffee_train):
    coffee_train_train = coffee_train.iloc[train_index]
    coffee_holdout = coffee_train.iloc[test_index]

    reg.fit(coffee_train_train[predictors], 
            coffee_train_train['rating'])
    
    preds = reg.predict(coffee_train_train[predictors])
    
    mse[i] = mean_squared_error(coffee_train_train['rating'], preds)
    mae[i] = mean_absolute_error(coffee_train_train['rating'], preds)
    
    i = i+1


In [22]:
## make the predictions for baseline
preds_baseline = baseline * np.ones(len(coffee_train))

In [23]:
## check the average mean squared error and compare to baseline
mse_baseline = mean_squared_error(coffee_train['rating'], preds_baseline)
print("The average cross validation mean squared error for multiple linear regression is", np.mean(mse))
print("The mean squared error for the baseline is", mse_baseline)

The average cross validation mean squared error for multiple linear regression is 9.924867305555471
The mean squared error for the baseline is 15.564935851389496


In [24]:
## check the average mean squared error and compare to baseline
mae_baseline = mean_absolute_error(coffee_train['rating'], preds_baseline)
print("The average cross validation mean absolute error for multiple linear regression is", np.mean(mae))
print("The mean absolute error for the baseline is", mae_baseline)

The average cross validation mean absolute error for multiple linear regression is 2.1024140116348797
The mean absolute error for the baseline is 2.854659147739123


Our MLR model appears to be performing well on the training set compared to the baseline model. 

## Running the model on the test split

Here we run the MLR model on the prior created test split.

In [14]:
reg.fit(coffee_train[predictors], coffee_train.rating)

LinearRegression()

In [15]:
pred = reg.predict(coffee_test[predictors])
test_mse = mean_squared_error(coffee_test.rating,pred)
test_mae = mean_absolute_error(coffee_test.rating,pred)

print("The average cross validation mean squared error for multiple linear regression is", test_mse)
print("The average cross validation mean absolute error for multiple linear regression is", test_mae)

The average cross validation mean squared error for multiple linear regression is 8.94406274648813
The average cross validation mean absolute error for multiple linear regression is 2.081188234508547


In [16]:
preds_baseline = baseline * np.ones(len(coffee_test))

mse_baseline = mean_squared_error(coffee_test['rating'], preds_baseline)
mae_baseline = mean_absolute_error(coffee_test['rating'], preds_baseline)
print("The mean squared error for the baseline is", mse_baseline)
print("The mean absolute error for the baseline is", mae_baseline)

The mean squared error for the baseline is 13.711274060947115
The mean absolute error for the baseline is 2.7539702122701315


## Saving the results

Now we save our test results.  The code below was run to create a csv file that contains the MSE and MAE for the baseline model and the MLR model. But it is commented out now as to not overwrite the created file.

In [17]:
#import csv

In [18]:
#with open('testing_results.csv', mode='w') as coffee_file:
#    results_writer = csv.writer(coffee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#    
#    results_writer.writerow(['Test', 'MSE', 'MAE'])
#    results_writer.writerow(['Baseline', mse_baseline, mae_baseline])
#    results_writer.writerow(['MLR', test_mse, test_mae])